# Battlefield Tester Explained

This notebook explains the comprehensive testing framework for analyzing battlefield performance.

## Overview

The tester runs multiple battles and generates detailed analytics:
- Victory distributions
- Win rates by team size
- Battle timelines  
- Initial configuration comparisons
- Comprehensive visualizations

In [1]:
import sys
sys.path.append('..')
sys.path.append('../testing_battlefield')

from battlefield_tester import BattlefieldTester
import quantum_library as qlib


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\adria\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\adria\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\adria\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\adria\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\adria\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\adria\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\adria\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\adria\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

## 1. Test Types

The framework provides 4 test types:

1. **Random Composition**: Completely random team compositions
2. **Balance Test**: Predefined balanced compositions
3. **Performance Test**: Mix of random and balanced for statistics
4. **Initial Configuration Test**: Tests different starting positions (Face to Face, Surrounded, Random)

## 2. Setting Up a Tester

In [ ]:
# Create a tester
tester = BattlefieldTester(
    num_battles=30,      # Number of battles to simulate
    max_turns=50,        # Max turns before declaring draw
    grid_size=(4, 4)     # Battlefield dimensions
)

print(f"Tester configured:")
print(f"  Battles: {tester.num_battles}")
print(f"  Max turns: {tester.max_turns}")
print(f"  Grid: {tester.grid_size}")

## 3. Generating Team Compositions

Compositions define team makeup: `{unit_type: (count, strength, health, range)}`

In [ ]:
# Random composition
random_comp = tester.generate_random_composition(max_units=8)
print("Random composition:")
for unit_type, (count, strength, health, range_dist) in random_comp.items():
    print(f"  {count}x {unit_type}: STR={strength}, HP={health}, RNG={range_dist}")

# Predefined balanced compositions
balanced_comps = tester.get_balanced_compositions()
print(f"\nAvailable balanced compositions: {len(balanced_comps)}")
for i, (q_comp, c_comp, desc) in enumerate(balanced_comps[:3]):
    print(f"{i+1}. {desc}")

## 4. Running Tests

### Test 1: Random Compositions

In [ ]:
# Set quantum method
qlib.set_quantum_method("exact")  # Use fast method for demo

# Run random composition test (small number for demo)
tester_random = BattlefieldTester(num_battles=10, max_turns=50, grid_size=(4, 4))
tester_random.test_random_compositions()

print(f"\nCompleted {len(tester_random.results)} battles")

### Test 4: Initial Configurations (NEW!)

In [ ]:
# Run initial configuration test
tester_config = BattlefieldTester(num_battles=9, max_turns=50, grid_size=(4, 4))
tester_config.test_initial_configurations()

# Analyze by configuration
configs = {}
for result in tester_config.results:
    config = result.initial_config
    if config not in configs:
        configs[config] = {'quantum': 0, 'classical': 0, 'draw': 0}
    
    if result.winner == 'Quantum':
        configs[config]['quantum'] += 1
    elif result.winner == 'Classical':
        configs[config]['classical'] += 1
    else:
        configs[config]['draw'] += 1

print("\nResults by configuration:")
for config, wins in configs.items():
    total = sum(wins.values())
    q_pct = wins['quantum'] / total * 100 if total > 0 else 0
    print(f"  {config}: Quantum {wins['quantum']}/{total} ({q_pct:.1f}%)")

## 5. Analyzing Results

Results are stored as `BattleResult` objects with detailed statistics.

In [ ]:
# Look at first result
if tester_config.results:
    result = tester_config.results[0]
    print(f"Battle {result.battle_id}:")
    print(f"  Winner: {result.winner}")
    print(f"  Duration: {result.turns} turns ({result.duration_seconds:.2f}s)")
    print(f"  Initial config: {result.initial_config}")
    print(f"  Quantum: {result.quantum_initial_count} → {result.quantum_survivors}")
    print(f"  Classical: {result.classical_initial_count} → {result.classical_survivors}")

## 6. Summary Statistics

In [ ]:
# Print summary
tester_config.print_summary_statistics()

## 7. Visualizations

The tester generates comprehensive dashboards with 6 plots:
1. Victory Distribution (pie chart)
2. Win Rate by Team Size
3. Battle Outcome Timeline
4. **Win Rate by Initial Configuration** (NEW!)
5. **Battle Duration by Configuration** (NEW!)
6. Summary Statistics Table

In [ ]:
# Generate plots (saves to file and displays)
tester_config.generate_comprehensive_plots()
# This will save to 'battlefield_test_results.png'

## 8. Command Line Usage

The tester can also be run from command line:

```bash
cd testing_battlefield

# Test 1: Random compositions
python battlefield_tester.py --test 1 --battles 100

# Test 2: Balanced compositions
python battlefield_tester.py --test 2 --battles 100

# Test 3: Performance metrics
python battlefield_tester.py --test 3 --battles 200

# Test 4: Initial configurations
python battlefield_tester.py --test 4 --battles 99

# Custom grid size
python battlefield_tester.py --test 4 --battles 99 --grid-width 6 --grid-height 6
```

In [ ]:
python battlefield_tester.py --test 1 --battles 100

## 9. Comparing EXACT vs QAOA

You can easily compare both quantum methods:

In [ ]:
# Test with EXACT method
qlib.set_quantum_method("exact")
tester_exact = BattlefieldTester(num_battles=10, max_turns=50, grid_size=(4, 4))
tester_exact.test_random_compositions()

quantum_wins_exact = sum(1 for r in tester_exact.results if r.winner == 'Quantum')
avg_turns_exact = sum(r.turns for r in tester_exact.results) / len(tester_exact.results)

print(f"EXACT Method Results:")
print(f"  Quantum wins: {quantum_wins_exact}/10 ({quantum_wins_exact*10}%)")
print(f"  Avg turns: {avg_turns_exact:.2f}")

# Test with QAOA method (WARNING: Will be slower!)
# Uncomment to run:
# qlib.set_quantum_method("qaoa")
# qlib.QAOA_P = 1
# qlib.QAOA_N_RESTARTS = 2
# tester_qaoa = BattlefieldTester(num_battles=10, max_turns=50, grid_size=(4, 4))
# tester_qaoa.test_random_compositions()
#
# quantum_wins_qaoa = sum(1 for r in tester_qaoa.results if r.winner == 'Quantum')
# avg_turns_qaoa = sum(r.turns for r in tester_qaoa.results) / len(tester_qaoa.results)
#
# print(f"\nQAOA Method Results (p=1, n_restarts=2):")
# print(f"  Quantum wins: {quantum_wins_qaoa}/10 ({quantum_wins_qaoa*10}%)")
# print(f"  Avg turns: {avg_turns_qaoa:.2f}")

## 10. Custom Analysis

You can access raw results for custom analysis:

In [ ]:
import numpy as np

if tester_exact.results:
    # Custom statistics
    turns = [r.turns for r in tester_exact.results]
    durations = [r.duration_seconds for r in tester_exact.results]
    
    print("Custom Statistics:")
    print(f"  Turn distribution: min={min(turns)}, max={max(turns)}, median={np.median(turns):.1f}")
    print(f"  Duration: min={min(durations):.2f}s, max={max(durations):.2f}s")
    
    # Survival rates
    quantum_survival = [r.quantum_survivors / r.quantum_initial_count 
                       for r in tester_exact.results if r.quantum_initial_count > 0]
    if quantum_survival:
        print(f"  Quantum avg survival rate: {np.mean(quantum_survival)*100:.1f}%")

## Summary

**Key Components**:
- `BattlefieldTester`: Main testing framework
- `BattleResult`: Stores individual battle statistics

**Test Types**:
1. Random Compositions
2. Balanced Compositions
3. Performance Metrics
4. Initial Configurations (Face to Face, Surrounded, Random)

**Outputs**:
- Console summary statistics
- Comprehensive 6-panel visualization dashboard
- Raw results for custom analysis

**Configuration**:
- Set quantum method in `quantum_library.py` or via `set_quantum_method()`
- Adjust QAOA parameters for quality/speed tradeoff
- See `QUANTUM_CONFIG.md` for detailed configuration guide

**Next Steps**:
- Run large-scale tests with `--battles 1000`
- Compare EXACT vs QAOA performance
- Analyze specific configurations
- Tune QAOA parameters for optimal results